# Welcome to Dataskillet!

In [1]:
import os
import modin.pandas as pd

In [2]:
from dataskillet import DataSource

In [3]:
%load_ext autoreload
%autoreload 2

Lets download some csvs

In [4]:
os.mkdir('testdrive_csvs')

FileExistsError: [Errno 17] File exists: 'testdrive_csvs'

In [ ]:
!curl https://raw.githubusercontent.com/jasonchang0/kaggle-google-apps/master/google-play-store-apps/googleplaystore.csv -o testdrive_csvs/googleplaystore.csv
!curl https://raw.githubusercontent.com/jasonchang0/kaggle-google-apps/master/google-play-store-apps/googleplaystore_user_reviews.csv -o testdrive_csvs/googleplaystore_user_reviews.csv

# The dataset contains google play apps and their user reviews about them

In [5]:
googleplaystore = pd.read_csv('testdrive_csvs/googleplaystore.csv')
googleplaystore.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [6]:
googleplaystore_user_reviews = pd.read_csv('testdrive_csvs/googleplaystore_user_reviews.csv')
googleplaystore_user_reviews.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


# Creating the DataSource

DataSource is the analogus of database. It stores information about tables and accepts queries to them.

On creation, we need to specify a directory to store table metadata for the DataSource.

In [7]:
metadata_dir = 'testdrive_metadata'
os.mkdir(metadata_dir)

FileExistsError: [Errno 17] File exists: 'testdrive_metadata'

In [8]:
ds = DataSource(metadata_dir=metadata_dir)

The metadata dir now stores information about tables. However we currently have no tables. Lets create some

In [9]:
ds.query('SHOW TABLES')

,name,fpath
0,googleplaystore,testdrive_csvs/googleplaystore.csv
1,googleplaystore_user_reviews,testdrive_csvs/googleplaystore_user_reviews.csv


# Creating tables

A `Table` is an abstraction over a dataframe. It loads the dataframe on-demand. On creation it applies simple preprocessings. The preprocessings are created once and stored in metadata, so they are always performed the same way when a dataframe is loaded.

In [10]:
ds.query(f'CREATE TABLE ("testdrive_csvs/googleplaystore.csv")')

Exception: Table googleplaystore already exists in data source, use DROP TABLE to remove it if you want to recreate it.

In [ ]:
ds.query(f'CREATE TABLE ("testdrive_csvs/googleplaystore_user_reviews.csv")')

In [11]:
ds.query('SHOW TABLES')

,name,fpath
0,googleplaystore,testdrive_csvs/googleplaystore.csv
1,googleplaystore_user_reviews,testdrive_csvs/googleplaystore_user_reviews.csv


Now that we have some tables, the information about them is stored in metadata. 

If we recreate the datasource using the same `metadata_dir`, we don't need to add the tables again.

In [12]:
ds = DataSource(metadata_dir=metadata_dir)

In [13]:
ds.query('SHOW TABLES')

,name,fpath
0,googleplaystore,testdrive_csvs/googleplaystore.csv
1,googleplaystore_user_reviews,testdrive_csvs/googleplaystore_user_reviews.csv


If we need it, we can clear the metadata and recreate the DataSource using `DataSource.create_new(metadata_dir)`

# Querying

In [14]:
ds.query('SELECT * FROM googleplaystore LIMIT 5')

,app,category,rating,reviews,size,installs,type,price,content_rating,genres,last_updated,current_ver,android_ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [15]:
ds.query("SELECT app FROM googleplaystore WHERE price = '0'")

0           Photo Editor & Candy Camera & Grid & ScrapBook
1                                      Coloring book moana
2        U Launcher Lite – FREE Live Cool Themes, Hide ...
3                                    Sketch - Draw & Paint
4                    Pixel Draw - Number Art Coloring Book
                               ...                        
10836                                     Sya9a Maroc - FR
10837                     Fr. Mike Schmitz Audio Teachings
10838                               Parkinson Exercices FR
10839                        The SCP Foundation DB fr nn5n
10840        iHoroscope - 2018 Daily Horoscope & Astrology
Name: app, Length: 10040, dtype: object

In [16]:
ds.query("SELECT * FROM googleplaystore WHERE category = 'FAMILY' AND price = '0' AND reviews > 5")

TypeError:  '>' not supported between instances of 'str' and 'int'

In [17]:
ds.query("SELECT app, sentiment, sentiment_polarity FROM googleplaystore INNER JOIN googleplaystore_user_reviews ON googleplaystore.app = googleplaystore_user_reviews.app LIMIT 10")

,app,sentiment,sentiment_polarity
0,Coloring book moana,Negative,-0.250
1,Coloring book moana,Negative,-0.725
2,Coloring book moana,Neutral,0.000
3,Coloring book moana,NaN,NaN
4,Coloring book moana,Positive,0.500
5,Coloring book moana,Negative,-0.800
6,Coloring book moana,NaN,NaN
7,Coloring book moana,Neutral,0.000
8,Coloring book moana,Positive,0.500
9,Coloring book moana,Positive,0.500


In [18]:
sql = """
SELECT app, avg(sentiment_polarity) as avg_sentiment_polarity
FROM (
    SELECT app, sentiment, sentiment_polarity 
    FROM googleplaystore INNER JOIN googleplaystore_user_reviews ON googleplaystore.app = googleplaystore_user_reviews.app 
) sub
GROUP BY app
LIMIT 10
"""
ds.query(sql)

To request implementation, send an email to feature_requests@modin.org.


,app,avg_sentiment_polarity
0,10 Best Foods for You,0.470733
1,11st,0.181294
2,1800 Contacts - Lens Store,0.318145
3,1LINE – One Line with One Touch,0.196290
4,2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif,0.449566
5,21-Day Meditation Experience,0.258014
6,"2Date Dating App, Love and matching",0.280267
7,2GIS: directory & navigator,0.223129
8,2RedBeans,0.412199
9,2ndLine - Second Phone Number,0.168968
